In [ ]:
import math
import pandas as pd
from flask import Flask, request, render_template_string, url_for
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

app = Flask(__name__)

# --------------------------------------------------------------------
# 1. LOAD DATA (combined_dataset.csv) AND TRAIN SVD MODEL
# --------------------------------------------------------------------
combined_df = pd.read_csv('combined_dataset.csv')

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(combined_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

model = SVD()
model.fit(trainset)

predictions = model.test(testset)
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)
print("MAE:", mae, "RMSE:", rmse)

# --------------------------------------------------------------------
# 2. HELPER FUNCTIONS
# --------------------------------------------------------------------
def get_user_recommendations(user_id, top_n=8):
    """
    Use the trained SVD model to recommend movies for a given user_id.
    1. Retrieve movies that the user has not rated.
    2. Predict the rating for each candidate movie and return top_n with the highest predicted rating.
    """
    all_movies = combined_df[['movieId', 'title']].drop_duplicates()
    user_rated = combined_df[combined_df['userId'] == user_id]['movieId'].unique()
    candidates = all_movies[~all_movies['movieId'].isin(user_rated)]
    
    pred_list = []
    for _, row in candidates.iterrows():
        iid = row['movieId']
        pred = model.predict(user_id, iid)
        pred_list.append((iid, pred.est, row['title']))
    pred_list.sort(key=lambda x: x[1], reverse=True)
    return pred_list[:top_n]

def get_unique_genres(df):
    """
    Extract all unique genres.
    Each movie's genres are stored in 'genres', separated by '|'.
    Filter out the value "(no genres listed)".
    """
    genres_set = set()
    for item in df['genres'].dropna():
        for g in item.split('|'):
            g_clean = g.strip()
            if g_clean and g_clean != "(no genres listed)":
                genres_set.add(g_clean)
    return sorted(genres_set)

def filter_movies_by_genre(df, genre=None):
    """
    If genre is 'Default' or not provided, return up to 300 movies.
    Otherwise, return movies whose 'genres' contain the selected genre.
    """
    if not genre or genre.lower() == 'default':
        return df.head(300).copy()
    else:
        return df[df['genres'].str.contains(genre, case=False, na=False)].copy()

def get_movie_ratings(df):
    """
    Compute average rating for each movie, returning DataFrame columns:
    movieId, title, genres, avg_rating.
    """
    grouped = df.groupby(['movieId', 'title', 'genres'], as_index=False)['rating'].mean()
    grouped.rename(columns={'rating': 'avg_rating'}, inplace=True)
    return grouped

def create_pagination_links(base_url, current_page, total_pages):
    """
    Generate HTML for pagination.
    All links include the "#movies" anchor and ajax=1 parameter.
    """
    page_links_html = []
    
    # Previous
    if current_page > 1:
        page_links_html.append(f'<a href="{base_url}page={current_page-1}#movies" class="ajax-link">&larr; Previous</a>')
    else:
        page_links_html.append('<span style="color:#ccc;">&larr; Previous</span>')
    
    # Page numbers
    if total_pages <= 5:
        pages_to_show = list(range(1, total_pages+1))
    else:
        pages_to_show = [1, 2, 3, 4]
        if current_page not in pages_to_show and current_page < total_pages - 2:
            pages_to_show.append('...')
        if current_page not in [1, 2, 3, 4, total_pages] and (1 < current_page < total_pages):
            pages_to_show.append(current_page)
        if total_pages not in pages_to_show:
            pages_to_show.append('...')
            pages_to_show.append(total_pages)
        new_list = []
        for p in pages_to_show:
            if p not in new_list:
                new_list.append(p)
        pages_to_show = sorted(new_list, key=lambda x: (x != '...', x))
    
    for p in pages_to_show:
        if p == '...':
            page_links_html.append('<span>...</span>')
        else:
            if p == current_page:
                page_links_html.append(
                    f'<span style="background-color:#000; color:#fff; padding:4px 8px; margin:0 2px;">{p}</span>'
                )
            else:
                page_links_html.append(f'<a href="{base_url}page={p}#movies" class="ajax-link">{p}</a>')
    
    # Next
    if current_page < total_pages:
        page_links_html.append(f'<a href="{base_url}page={current_page+1}#movies" class="ajax-link">Next &rarr;</a>')
    else:
        page_links_html.append('<span style="color:#ccc;">Next &rarr;</span>')
    
    return ' '.join(page_links_html)

movie_ratings_df = get_movie_ratings(combined_df)
all_genres = get_unique_genres(combined_df)
all_genres = ['Default'] + all_genres

# --------------------------------------------------------------------
# 3. ROUTES
# --------------------------------------------------------------------
@app.route('/')
def home():
    """
    Home page with:
      - Divider, carousel, login and genre links, movies grid, and pagination.
      - If parameter ajax=1 is set, return only the movie grid and pagination fragment.
    """
    genre = request.args.get('genre', default='Default')
    # Base URL for ajax links (include ajax=1)
    base_url = f"/?genre={genre}&ajax=1&"
    
    filtered_movies = filter_movies_by_genre(movie_ratings_df, genre)
    filtered_movies = filtered_movies.sort_values(by='avg_rating', ascending=False)
    
    page = request.args.get('page', default=1, type=int)
    items_per_page = 8  # fewer per page so each item is bigger
    total_items = len(filtered_movies)
    total_pages = math.ceil(total_items / items_per_page)
    start_idx = (page - 1) * items_per_page
    end_idx = start_idx + items_per_page
    page_movies = filtered_movies.iloc[start_idx:end_idx]
    
    pagination_html = create_pagination_links(base_url, page, total_pages)
    
    # Check if this is an AJAX request
    is_ajax = request.args.get('ajax') == '1'
    
    # Define movie grid and pagination fragment
    movies_fragment = '''
<div id="movies">
  <div class="movies-grid">
    {% for idx, row in page_movies.iterrows() %}
      <div class="film-frame">
        <div class="film-text">
          <p>Movie ID: {{ row.movieId }}</p>
          <p>Movie Title: {{ row.title }}</p>
          <p>Average Rating: {{ "%.2f"|format(row.avg_rating) }}</p>
        </div>
      </div>
    {% endfor %}
  </div>
  <div class="pagination" style="margin-bottom: 30px;">
    {{ pagination_html|safe }}
  </div>
</div>
    '''
    if is_ajax:
        # Return only the movies fragment
        return render_template_string(movies_fragment,
                                      page_movies=page_movies,
                                      pagination_html=pagination_html)
    
    # Otherwise, return full page
    full_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Movie Recommendation</title>
  <link href="https://fonts.googleapis.com/css2?family=Varela+Round&display=swap" rel="stylesheet">
  <style>
    /* Global */
    body {
      margin: 0;
      padding: 0;
      font-family: 'Varela Round', sans-serif;
      color: #595959;
      background-color: #f8f8f8;
    }
    h1 {
      text-align: center;
      font-size: 36px;
      margin-top: 80px;
      margin-bottom: 40px;
    }
    .divider {
      width: 100%;
      height: 1px;
      background-color: #ccc;
      margin: 40px 0;
    }
    .container {
      max-width: 1200px;
      margin: 0 auto;
      text-align: center;
      padding: 0 20px 40px;
    }
    /* Carousel */
    .carousel-container {
      position: relative;
      width: 100%;
      max-width: 1200px;
      margin: 0 auto;
    }
    .carousel {
      overflow: hidden;
    }
    .carousel-images {
      display: flex;
      transition: transform 0.5s ease-in-out;
    }
    .carousel-images img {
      width: 100%;
      object-fit: cover;
      border-radius: 10px;
    }
    .carousel-arrow {
      position: absolute;
      top: 50%;
      transform: translateY(-50%);
      background-color: rgba(105,105,105,0.8);
      color: #fff;
      border: none;
      padding: 10px;
      cursor: pointer;
      border-radius: 10px;
      z-index: 10;
    }
    .carousel-arrow.left {
      left: -80px;
    }
    .carousel-arrow.right {
      right: -80px;
    }
    .carousel-indicators {
      margin-top: 10px;
    }
    .carousel-indicators span {
      display: inline-block;
      width: 12px;
      height: 12px;
      background-color: #ccc;
      margin: 0 5px;
      border-radius: 50%;
      cursor: pointer;
    }
    .carousel-indicators span.active {
      background-color: #595959;
    }
    /* LOGIN SECTION */
    .login-section {
      text-align: center;
      margin-top: 40px;
      margin-bottom: 40px;
    }
    .login-section h2 {
      font-size: 24px;
      margin-bottom: 20px;
    }
    .login-section input[type="text"] {
      padding: 16px;
      font-size: 18px;
      border: 1px solid #ccc;
      border-radius: 10px;
      width: 300px;
    }
    .login-section input[type="submit"] {
      padding: 16px 28px;
      font-size: 18px;
      border: none;
      border-radius: 10px;
      background-color: #595959;
      color: #fff;
      cursor: pointer;
      margin-left: 10px;
    }
    .login-section input[type="submit"]:hover {
      background-color: #444;
    }
    /* GENRES */
    .genres {
      text-align: center;
      margin-bottom: 20px;
    }
    .genres a {
      margin: 0 6px;
      text-decoration: none;
      color: #595959;
      font-size: 14px;
      background-color: #f0f0f0;
      padding: 6px 10px;
      border-radius: 4px;
    }
    .genres a:hover {
      background-color: #e0e0e0;
    }
    /* MOVIES GRID */
    .movies-grid {
      display: grid;
      grid-template-columns: repeat(4, 1fr);
      gap: 20px;
      margin-top: 20px;
      justify-items: center;
    }
    /* Enlarged  */
    .film-frame {
      background: url('{{ url_for('static', filename='frame.webp') }}') no-repeat center center;
      background-size: contain;
      width: 95%;
      max-width: 450px;
      aspect-ratio: 4 / 3;
      border-radius: 50px;
      overflow: hidden;
      display: flex;
      justify-content: center;
      align-items: center;
      text-align: center;
      padding: 70px;
      box-sizing: border-box;
    }
    .film-text {
      max-width: 90%;
      margin: 0 auto;
      color: #000;
    }
    .film-text p {
      margin: 8px 0;
      font-size: 18px;
    }
    /* PAGINATION with extra bottom margin */
    .pagination {
      text-align: center;
      margin-top: 40px;
      margin-bottom: 30px;
    }
    .pagination a, .pagination span {
      margin: 0 3px;
      text-decoration: none;
      color: #000;
    }
  </style>
</head>
<body>
  <!-- PAGE TITLE & DIVIDER -->
  <h1>Movie Recommendation</h1>
  <div class="divider"></div>

  <!-- CAROUSEL & INDICATORS -->
  <div class="container">
    <div class="carousel-container">
      <div class="carousel">
        <div class="carousel-images">
          <img src="{{ url_for('static', filename='black_mirror.jpg') }}" alt="Movie Image 1">
          <img src="{{ url_for('static', filename='frozen.png') }}" alt="Movie Image 2">
          <img src="{{ url_for('static', filename='iceage.jpg') }}" alt="Movie Image 3">
          <img src="{{ url_for('static', filename='toy.webp') }}" alt="Movie Image 4">
        </div>
      </div>
      <button class="carousel-arrow left">&#9664;</button>
      <button class="carousel-arrow right">&#9654;</button>
    </div>
    <div class="carousel-indicators">
      <span class="active" data-index="0"></span>
      <span data-index="1"></span>
      <span data-index="2"></span>
      <span data-index="3"></span>
    </div>
  </div>
  
  <!-- DIVIDER BETWEEN CAROUSEL AND LOGIN -->
  <div class="divider"></div>

  <!-- LOGIN SECTION -->
  <div class="login-section">
    <h2>Login for personal recommendation</h2>
    <form action="/login" method="post">
      <input type="text" name="user_id" placeholder="Enter your User ID" required>
      <input type="submit" value="search">
    </form>
  </div>

  <!-- GENRES -->
  <div class="genres">
    Genres:
    {% for g in all_genres %}
      <a href="/?genre={{ g }}&ajax=1#movies" class="ajax-link">{{ g }}</a>
    {% endfor %}
  </div>

  <!-- MOVIES GRID & PAGINATION -->
  ''' + movies_fragment + '''
  
  <!-- AJAX SCRIPT -->
  <script>
    // Execute carousel script when DOM is fully loaded
    document.addEventListener("DOMContentLoaded", function() {
      const carouselImages = document.querySelector('.carousel-images');
      const images = document.querySelectorAll('.carousel-images img');
      const leftArrow = document.querySelector('.carousel-arrow.left');
      const rightArrow = document.querySelector('.carousel-arrow.right');
      const indicators = document.querySelectorAll('.carousel-indicators span');
      let currentIndex = 0;
      const totalImages = images.length;
      
      function updateCarousel() {
        carouselImages.style.transform = 'translateX(-' + (currentIndex * 100) + '%)';
        indicators.forEach((indicator, index) => {
          if (index === currentIndex) {
            indicator.classList.add('active');
          } else {
            indicator.classList.remove('active');
          }
        });
      }
      rightArrow.addEventListener('click', () => {
        currentIndex = (currentIndex + 1) % totalImages;
        updateCarousel();
      });
      leftArrow.addEventListener('click', () => {
        currentIndex = (currentIndex - 1 + totalImages) % totalImages;
        updateCarousel();
      });
      indicators.forEach(indicator => {
        indicator.addEventListener('click', () => {
          currentIndex = Number(indicator.getAttribute('data-index'));
          updateCarousel();
        });
      });
      setInterval(() => {
        currentIndex = (currentIndex + 1) % totalImages;
        updateCarousel();
      }, 5000);
    });
    
    // AJAX handling for pagination and genre links
    document.addEventListener("DOMContentLoaded", function() {
      function ajaxifyLinks() {
        document.querySelectorAll("a.ajax-link").forEach(link => {
          link.addEventListener("click", function(e) {
            e.preventDefault();
            const url = link.getAttribute("href");
            fetch(url)
              .then(response => response.text())
              .then(html => {
                document.getElementById("movies").innerHTML = html;
                // Update browser URL (remove ajax=1 if desired)
                history.pushState(null, "", url.replace("&ajax=1", ""));
                // Re-bind ajax click events for new links
                ajaxifyLinks();
              })
              .catch(err => console.error(err));
          });
        });
      }
      ajaxifyLinks();
    });
  </script>
</body>
</html>
    '''
    return render_template_string(full_template,
                                  all_genres=all_genres,
                                  page_movies=page_movies,
                                  pagination_html=pagination_html,
                                  genre=genre)

@app.route('/login', methods=['POST'])
def login():
    """
    Accept a user ID and return personalized recommendations with 4 columns,
    enlarged film-frame as above.
    """
    user_id = request.form.get('user_id')
    if not user_id:
        return "User ID cannot be empty."
    try:
        user_id_int = int(user_id)
    except ValueError:
        return "Invalid User ID; it must be a number."
    
    recs = get_user_recommendations(user_id_int, top_n=8)
    html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Recommendations for User {{ user_id }}</title>
  <link href="https://fonts.googleapis.com/css2?family=Varela+Round&display=swap" rel="stylesheet">
  <style>
    body {
      margin: 0;
      padding: 0;
      font-family: 'Varela Round', sans-serif;
      color: #595959;
      background-color: #f8f8f8;
    }
    h1 {
      text-align: center;
      margin-top: 60px;
      font-size: 32px;
    }
    .divider {
      width: 100%;
      height: 1px;
      background-color: #ccc;
      margin: 30px 0;
    }
    .container {
      max-width: 1200px;
      margin: 0 auto;
      padding: 20px 40px 80px;
      text-align: center;
    }
    .recommendation-grid {
      display: grid;
      grid-template-columns: repeat(4, 1fr);
      gap: 30px;
      margin-top: 30px;
      justify-items: center;
    }
    .film-frame {
      background: url('{{ url_for('static', filename='frame.webp') }}') no-repeat center center;
      background-size: contain;
      width: 100%;
      max-width: 950px;
      aspect-ratio: 4 / 3;
      border-radius: 35px;
      overflow: hidden;
      display: flex;
      justify-content: center;
      align-items: center;
      text-align: center;
      padding: 30px;
      box-sizing: border-box;
      margin: 0 auto;
    }
    .film-text {
      max-width: 90%;
      margin: 0 auto;
    }
    .film-text p {
      margin: 8px 0;
      font-size: 18px;
    }
    .return-home {
      display: inline-block;
      margin-top: 80px;
      padding: 30px 60px;
      background-color: #7B7B7B;
      color: #fff;
      border: none;
      border-radius: 10px;
      text-decoration: none;
      font-size: 24px;
    }
    .return-home:hover {
      background-color: #444;
    }
  </style>
</head>
<body>
  <h1>Recommendations for User {{ user_id }}</h1>
  <div class="divider"></div>
  <div class="container">
    <div class="recommendation-grid">
      {% for rec in recs %}
      <div class="film-frame">
        <div class="film-text">
          <p>Movie ID: {{ rec[0] }}</p>
          <p>Movie Title: {{ rec[2] }}</p>
          <p>Average Rating: {{ "%.2f"|format(rec[1]) }}</p>
        </div>
      </div>
      {% endfor %}
    </div>
    <a href="/" class="return-home">Return to Home</a>
  </div>
</body>
</html>
    '''
    return render_template_string(html_template, user_id=user_id, recs=recs)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=True, use_reloader=False)


MAE:  0.6765
RMSE: 0.8793
MAE: 0.6765208388394469 RMSE: 0.8793074605706716
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.50.20:5001
Press CTRL+C to quit
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET / HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET /static/black_mirror.jpg HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET /static/iceage.jpg HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET /static/frozen.png HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET /static/toy.webp HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:06:55] "GET /static/frame.webp HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:11:44] "GET /?genre=Crime&ajax=1 HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:13:07] "POST /login HTTP/1.1" 200 -
192.168.50.20 - - [24/Apr/2025 21:13:07] "GET /static/frame.webp HTTP/1.1" 304 -
